## 本人在学习迁移学习，使用Keras中的预训练模型（Pretrained Model）时，积累的一些Tricks

### 1.Preappend a layer to a model

In [ ]:
import tensorflow as tf

input_t = tf.keras.Input(shape=(224, 224, 1))
conv1 = tf.keras.layers.Conv2D( filters=3, 
                                kernel_size=(1, 1), 
                                padding='same',
                                name='conv1',
                                kernel_initializer=tf.glorot_normal_initializer())(input_t)
    
bn1 = tf.keras.layers.BatchNormalization(fused=True, name='bn1')(conv1)
resnet_output = tf.keras.applications.MobileNet()(bn1)

model = tf.keras.models.Model(input_t, resnet_output)

### 2. Some tricks on image processing

In [ ]:
# image size from (256, 256) to (1, 256, 256)
image = np.expand_dims(image, axis=0)

# image size from (1, 256, 256) to (1, 256, 256, 1)
image = np.expand_dims(image, axis=3)

# Grayscale_To_RGB, image: dtype is array
image = (image+1)*127
uint_image = image.astype(np.uint8) # important convert pixel type from float to uint
gray3_image = np.repeat(uint_image, 3, axis=-1)

### 3.PB 文件转成graph

In [ ]:
## PB file 可视化，转成log文件，通过tensorboard显示
import tensorflow as tf
from tensorflow.python.platform import gfile
with tf.Session() as sess:
    model_filename ='./loglog/expert-graph.pb'
    with gfile.FastGFile(model_filename, 'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
        g_in = tf.import_graph_def(graph_def)
LOGDIR='./mypblog'
train_writer = tf.summary.FileWriter(LOGDIR)
train_writer.add_graph(sess.graph)

### 4.加载PB文件

In [ ]:
def load_graph(model_file):
    graph = tf.Graph()
    graph_def = tf.GraphDef()

    with open(model_file, "rb") as f:
        graph_def.ParseFromString(f.read())
    with graph.as_default():
        tf.import_graph_def(graph_def)
        
    return graph

model_file = './models/expert-graph.pb'
graph = load_graph(model_file)

input_op = graph.get_operation_by_name('import/input_1')
output_op = graph.get_operation_by_name('import/dense_1')

### 5.灰度图去噪

In [ ]:
def denoise(img, weight=0.15, eps=1e-3, num_iter_max=200):
    u = np.zeros_like(img)
    px = np.zeros_like(img)
    py = np.zeros_like(img)

    nm = np.prod(img.shape[:2])
    tau = 0.125

    i = 0
    while i < num_iter_max:
        u_old = u
        
        # x and y components of u's gradient
        ux = np.roll(u, -1, axis=1) - u
        uy = np.roll(u, -1, axis=0) - u

        # update the dual variable
        px_new = px + (tau / weight) * ux
        py_new = py + (tau / weight) * uy

        norm_new = np.maximum(1, np.sqrt(px_new **2 + py_new ** 2))
        px = px_new / norm_new
        py = py_new / norm_new

        # calculate divergence
        rx = np.roll(px, 1, axis=1)
        ry = np.roll(py, 1, axis=0)
        div_p = (px - rx) + (py - ry)

        # update image
        u = img + weight * div_p

        # calculate error
        error = np.linalg.norm(u - u_old) / np.sqrt(nm)
    
        if i == 0:
            err_init = error
            err_prev = error
        else:
            # break if error small enough
            if np.abs(err_prev - error) < eps * err_init:
                break
            else:
                e_prev = error

        # don't forget to update iterator
        i += 1

    return u